In [1]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

# Data Extraction

We'll use this notebook to create the datasets we are going to use.

All the datasets created here will be saved in data directory

In [2]:
path_to_data = Path("../data")  # Access to data folder

### All features

We start by extracting all the features from the table given from the professor

In [ ]:
# Combine all PDBs into a single dataframe
ring_path = Path(
    "/Users/mattiapiazza/Documents/University/structural_bioinformatics/project/sb_project/features_ring"
)

dfs = []
for filename in os.listdir(ring_path):
    dfs.append(pd.read_csv(ring_path / filename, sep="\t"))
df_complete = pd.concat(dfs)
df_complete.head()

In [ ]:
# Saving the dataset
df_complete.to_csv(path_to_data / "df_complete.csv")